# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(["127.0.0.1"])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("udacity")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Verify if anticipated primary key (sessionId + itemInSession) makes the row unique
df = pd.read_csv("event_datafile_new.csv")
df[df["sessionId"] == 338]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
441,Pixies,Ava,F,1,Robinson,89.36444,free,"New Haven-Milford, CT",338,Build High,50
442,The Roots / Jack Davey,Ava,F,2,Robinson,155.95057,free,"New Haven-Milford, CT",338,Atonement,50
443,Mike And The Mechanics,Ava,F,3,Robinson,275.12118,free,"New Haven-Milford, CT",338,A Beggar On A Beach Of Gold,50
444,Faithless,Ava,F,4,Robinson,495.30730,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_query_session_item = "CREATE TABLE IF NOT EXISTS song_library"
create_query_session_item = create_query_session_item + """(
sessionid int, iteminsession int, artist text, song text, length float, 
PRIMARY KEY (sessionid, iteminsession)
)"""

try:
    session.execute(create_query_session_item)
except Exception as e:
    print(e)

In [10]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        insert_query_session_item = "INSERT INTO song_library (sessionid, iteminsession, artist, song, length)"
        insert_query_session_item = insert_query_session_item + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insert_query_session_item, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
### For table song_library, the columns sessionid and iteminsession were used as a partition key because the queries will filter by these columns and also these columns make up a unique key."""

select_query_session_item = "SELECT artist, song, length FROM song_library WHERE sessionid = 338 AND iteminsession = 4"
try:
    rows = session.execute(select_query_session_item)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, row.song, row.length)   

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## Create table
create_query_session_user = "CREATE TABLE IF NOT EXISTS artist_library"
create_query_session_user = create_query_session_user + """(
userid int, sessionid int, iteminsession int, artist text, song text, firstname text, lastname text, 
PRIMARY KEY ((userid, sessionid), iteminsession)
)"""

try:
    session.execute(create_query_session_user)
except Exception as e:
    print(e)

In [13]:
## Insert data
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        insert_query_session_user = "INSERT INTO artist_library (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        insert_query_session_user = insert_query_session_user + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insert_query_session_user, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [14]:
### For table artist_library, the columns sessionid and userid were used as a partition key because the queries will filter by these columns and also these columns make up a unique key. iteminsession column was used as clustering column since we needed to sort results by this column.

## Select query
select_query_session_user = "SELECT artist, song, firstname, lastname FROM artist_library WHERE sessionid = 182 AND userid = 10"
try:
    rows = session.execute(select_query_session_user)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)   

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [15]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Create table
create_query_song_user = "CREATE TABLE IF NOT EXISTS user_library "
create_query_song_user = create_query_song_user + """(song text, userid int, firstname text, lastname text, 
    PRIMARY KEY (song, userid))"""
try:
    session.execute(create_query_song_user)
except Exception as e:
    print(e)         

In [16]:
## Insert data
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        insert_query_song_user = "INSERT INTO user_library (song, userid, firstname, lastname)"
        insert_query_song_user = insert_query_song_user + " VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insert_query_song_user, (line[9], int(line[10]), line[1], line[4]))

In [17]:
### For table user_library, the columns song and userid were used as a partition key because they make up a unique key and query filters on song.

## Select query
select_query_song_user  = """SELECT firstname, lastname
                    FROM user_library 
                    WHERE song = 'All Hands Against His Own'"""
try:
    rows = session.execute(select_query_song_user)
except Exception as e:
    print(e)

for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
## Drop the table before closing out the sessions

delete_song_library = "drop table song_library"
try:
    rows = session.execute(delete_song_library)
except Exception as e:
    print(e)

delete_artist_library = "drop table artist_library"
try:
    rows = session.execute(delete_artist_library)
except Exception as e:
    print(e)

delete_user_library = "drop table user_library"
try:
    rows = session.execute(delete_user_library)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()